In [1]:
import tensorflow as tf
import librosa
import numpy as np
import IPython.display as ipd
import os
import pandas as pd

In [2]:
X_0 = []
paths = librosa.util.find_files(r'orig\Без шума\Старт')
for i in paths:
    x, sr = librosa.load(i)
    mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=512, n_mfcc=20)
    mfcc = np.mean(mfcc, axis=1)
    X_0.append(mfcc)
np.shape(X_0)

(71, 20)

In [3]:
y_0 = np.zeros(np.shape(X_0)[0])

In [11]:
X_1 = []
voices_path = r'C:\Users\misai\Программирование\Проекты\PhraseRecognition\audiobooks\data\ru_open_stt\private_buriy_audiobooks_2\0'
for path in os.listdir(voices_path):
    paths = librosa.util.find_files(voices_path + '\\' + path)
    for file in paths:
        x, sr = librosa.load(file)
        mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=512, n_mfcc=20)
        mfcc = np.mean(mfcc, axis=1)
        X_1.append(mfcc)
np.shape(X_1)

(3359, 20)

In [30]:
y_1 = np.ones(3359)

In [14]:
save = np.copy(X_1)

In [6]:
columns = list('mfcc' + str(i) for i in range(1, 21))

In [7]:
DF_0 = pd.DataFrame(X_0, columns=list('mfcc' + str(i) for i in range(1, 21)))

In [29]:
DF_1 = pd.DataFrame(X_1, columns=list('mfcc' + str(i) for i in range(1, 21)))

In [8]:
save_1 = pd.read_csv(r'C:\Users\misai\Программирование\Проекты\PhraseRecognition\DF_1.csv', index_col=0)

In [82]:
X = DF_0.append(save_1, ignore_index=True)
X

(3430, 20)

In [31]:
y = np.append(y_0, y_1)
y

array([0., 0., 0., ..., 1., 1., 1.])

In [69]:
y = pd.DataFrame(y)

In [84]:
y

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3425,1.0
3426,1.0
3427,1.0
3428,1.0


In [62]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(20,)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(40, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])

In [63]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
dataset = tf.data.Dataset.from_tensor_slices((X.values, y.values))

In [37]:
shuffled_dataset = dataset.shuffle(len(X)).batch(1)

In [38]:
val_dataset = shuffled_dataset.take(1000) 
train_dataset = shuffled_dataset.skip(1000)

In [64]:
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
2430/2430 [==============================] - 8s 3ms/step - loss: 0.3317 - accuracy: 0.9794 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
2430/2430 [==============================] - 4s 2ms/step - loss: 0.3318 - accuracy: 0.9794 - val_loss: 0.4030 - val_accuracy: 0.9750
Epoch 3/10
2430/2430 [==============================] - 4s 2ms/step - loss: 0.3384 - accuracy: 0.9790 - val_loss: 0.2579 - val_accuracy: 0.9840
Epoch 4/10
2430/2430 [==============================] - 5s 2ms/step - loss: 0.3384 - accuracy: 0.9790 - val_loss: 0.3385 - val_accuracy: 0.9790
Epoch 5/10
2430/2430 [==============================] - 5s 2ms/step - loss: 0.3052 - accuracy: 0.9811 - val_loss: 0.2579 - val_accuracy: 0.9840
Epoch 6/10
2430/2430 [==============================] - 5s 2ms/step - loss: 0.3185 - accuracy: 0.9802 - val_loss: 0.2740 - val_accuracy: 0.9830
Epoch 7/10
2430/2430 [==============================] - 5s 2ms/step - loss: 0.3384 - accuracy: 0.9790 - val_loss: 0.3224 - val_a

In [67]:
model.predict(X_0[32].reshape(1, 20))

array([[0., 1.]], dtype=float32)

In [60]:
X_0[32]

array([-301.46695   ,  102.895096  ,    7.2665424 ,   33.10264   ,
          2.9483023 ,    4.9492803 ,  -14.442509  ,    3.605436  ,
        -16.99623   ,    5.7312164 ,   -1.5544255 ,    0.65738374,
         -6.0706964 ,   -2.1306076 ,   -5.927425  ,    5.81626   ,
        -10.802134  ,   -1.5740451 ,   -6.3382063 ,   -3.1696377 ],
      dtype=float32)